In [1]:
# LOAD DATASET 

# TODO: reuse tokenized_dataset.py:load_wmt14_load_dataset?
import datasets
from datasets import load_dataset
ds = load_dataset("wmt14", "de-en", streaming=True) # 5.4mln rows
ds = ds.map(lambda item: {'x':item['translation']["en"], 'y':item['translation']["de"]}, remove_columns=["translation"], batched=False)

next(x for x in ds["train"])

Loading dataset


TypeError: IterableDatasetDict.map() got an unexpected keyword argument 'num_proc'

In [ ]:
# BUILD/LOAD (test) TOKENIZER
# Load functionality is just for testing (for efficient runs, use load_dataset_and_load_tokenizer.py instead )
from tokenizer import *
import itertools

TOK_DS_SIZE = 1000
BPE_NUM_MERGES = 1000

#(tokenize, detokenize), state = build_w_tokenizer(itertools.islice(ds["train"], TOK_DS_SIZE))
#(tokenize, detokenize), state = build_bpe_tokenizer(itertools.islice(ds["train"], TOK_DS_SIZE), BPE_NUM_MERGES)


# building
#(tokenize, detokenize), state = build_bpe_tokenizer(itertools.islice(ds["train"], 100), 10) #debugging
#(tokenize, detokenize), state = build_bpe_tokenizer(itertools.islice(ds["train"], 100000), 10000)

#(tokenize, detokenize), state = build_bpe_tokenizer(ds["train"], 10000) # Runs in under 2hours
#save_tokenizer(state, "bpe_tokenizer_ds_train_all_merges_10k.pickle") # 13k vocab size

(tokenize, detokenize), state = build_bpe_tokenizer(ds["train"], 35000) # XXX: TODO add time
save_tokenizer(state, "bpe_tokenizer_ds_train_all_merges_35k.pickle") # XXX: TODO add size

# loading
#(tokenize, detokenize), state = load_bpe_tokenizer("bpe_tokenizer_ds_100k_merges_10k_fixed.pickle")
#(tokenize, detokenize), state = load_bpe_tokenizer("bpe_tokenizer_ds_train_all_merges_10k.pickle")

for item in itertools.islice(ds["train"], 1):
    #print(f'x {item["x"]} tokenized {tokenize([item["x"]])} detokenized {detokenize(tokenize([item["x"]]))}')
    x_and_y = [item["x"], item["y"]]
    print(f'x_and_y {x_and_y} tokenized {tokenize(x_and_y)} detokenized {detokenize(tokenize(x_and_y))}')

vocab_map = state[0][0] # we need this line for t.ipynb notebook
print(len(vocab_map))

Build BPE-ified word vocabulary


4508785it [03:48, 19701.62it/s]


len(vocab) 3946607
Establish 35000 BPE merges


 84%|████████▎ | 29272/35000 [9:59:39<2:57:07,  1.86s/it]

In [6]:
# Data collator
#%load_ext line_profiler
from tqdm import tqdm
    
def get_batched_examples(batch_size, max_len, split="train"):
    def encode_batch(batch, max_len):
        batch_str = [ x for x, _ in batch] + [ y for _, y in batch]
        pad = lambda toks: toks + [0] * (max_len - len(toks))
        encoded = [pad(str)[:max_len] for str in tokenize(batch_str)]
        return zip(encoded[:int(len(encoded)/2)], encoded[int(len(encoded)/2):])
    
    batch = []
    for item in ds[split]:
        batch.append((item['x'],item['y']))
        if len(batch) == batch_size:
            yield zip(*encode_batch(batch, max_len))
            batch = []
    if len(batch)>0:
        yield zip(*encode_batch(batch))

for x, y in tqdm(itertools.islice(get_batched_examples(3, 5), 1)):
    print(f'x {x}')
    print(f'y {y}')

#%lprun -f batched_apply_bpe_merges4 hello()

1it [00:00, 226.23it/s]

x ([3588, 3691, 12118, 3613, 3586], [3667, 4531, 19, 3715, 3588], [3585, 10333, 3578, 3626, 3829])
y ([5476, 10343, 3606, 6056, 11623], [3638, 5706, 3567, 3610, 3979], [3869, 3725, 4824, 5435, 6539])
